# Détection en Temps Réel

Ce notebook teste la détection de panneaux en temps réel avec une webcam.

## Objectifs
- Charger un modèle entraîné
- Détecter les panneaux dans une image
- Tester avec la webcam (optionnel)


In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent / "src"))

import numpy as np
import matplotlib.pyplot as plt
import cv2
from src.data_loader import GTSRBDataLoader
from src.detector import RealTimeDetector
from src.model import TrafficSignClassifier

DATA_PATH = "../data"
MODEL_PATH = "../models/traffic_sign_cnn.h5"


## 1. Charger le Modèle


In [ ]:
# Vérifier si le modèle existe
from pathlib import Path
model_file = Path(MODEL_PATH)

if not model_file.exists():
    print(f"⚠️ Le modèle {MODEL_PATH} n'existe pas!")
    print("Entraînez d'abord un modèle avec le notebook 03_model_training.ipynb")
else:
    print(f"✅ Modèle trouvé: {MODEL_PATH}")
    
    # Charger les noms de classes
    loader = GTSRBDataLoader(DATA_PATH)
    class_names = loader.get_class_names()
    
    # Créer le détecteur
    detector = RealTimeDetector(MODEL_PATH, class_names)
    print("✅ Détecteur initialisé")


## 2. Test sur une Image

Testez la détection sur une image statique.


In [ ]:
# Charger une image de test
test_image_path = "../data/Test/00000.ppm"  # Modifiez selon votre dataset

from pathlib import Path
if Path(test_image_path).exists():
    result = detector.detect_from_image(test_image_path)
    
    # Afficher le résultat
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title("Détection sur Image")
    plt.axis('off')
    plt.show()
else:
    print(f"Image de test non trouvée: {test_image_path}")
    print("Utilisez une image de votre dataset de test")


## 3. Détection en Temps Réel avec Webcam

⚠️ **Note**: Exécutez cette cellule pour lancer la détection en temps réel. Appuyez sur 'q' pour quitter.


In [ ]:
# Lancer la détection en temps réel
# Décommentez la ligne suivante pour activer la webcam
# detector.run_detection(camera_index=0)
